### 1. Data Preprocessing

In [23]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from itertools import combinations
from collections import defaultdict

import pickle
import csv

In [4]:
recipe_cat = pd.read_csv('data/레시피_분류_내용_raw.csv')
recipe_cat.sample(3)

,레시피_아이디,카테고리_코드,카테고리_내용,분류_코드,분류_내용
523454,6931538,cat4,종류별,66,빵
587193,6948163,cat3,재료별,48,과일류
332121,6880221,cat3,재료별,34,기타


#### 분류 코드 임베딩
총 61가지의 고유한 정수값을 가지는 분류 코드(e.g. 종류별-밥/죽/떡 = 52)를 벡터로 표현하기 위해 임베딩을 진행했다.
- 각 분류 코드를 오름차순으로 정렬 → 이를 index로 mapping → 그 index값을 lookup table의 index로 사용
- 임베딩 차원은 5

In [5]:
catList = sorted(list(recipe_cat.분류_코드.unique()))
cat2idx = {v:k for k,v in enumerate(catList)}

catEmb = nn.Embedding(61,5)

# 분류 코드가 0,1,2,3,4,..로 넘버링 된 게 아니라는 점에 유의!!!
catTensorList = [torch.LongTensor([cat2idx[elem]]) for elem in catList]

catEmbList = [catEmb(elem) for elem in catTensorList]
catEmbList[:3]

[tensor([[-0.4072, -0.1621, -0.3144, -0.7836,  1.1554]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[-1.0333,  0.1303, -1.1901, -2.6433,  0.3622]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[ 1.7870, -1.2331, -1.7418, -0.1117,  2.2956]],
        grad_fn=<EmbeddingBackward0>)]

#### 레시피의 분류 코드 합치기
각 레시피는 여러 분류 코드를 가질 수 있으므로 이를 한 번에 담아두기로 했다.

In [6]:
recipe_cat_copy = recipe_cat[['레시피_아이디','분류_코드']]
recipe_cat_list = recipe_cat_copy.groupby('레시피_아이디').agg(lambda x:sorted(x))
recipe_cat_list

,분류_코드
레시피_아이디,
13654,"[11, 17, 28, 59]"
13655,"[1, 17, 24, 68]"
13656,"[11, 17, 34, 52]"
13657,"[11, 17, 27, 60]"
13998,"[1, 17, 28, 68]"
...,...
6979767,"[1, 24, 46, 54]"
6979768,"[43, 56, 67, 71]"
6979769,"[11, 21, 48, 64]"


#### 특정 레시피 데이터만 사용하기
전체 레시피는 약 17만 개이므로 너무 많아 리뷰 수가 7개 이상, 조회수가 20000회가 넘는 레시피만 사용한다.
- 즉, 약 8000개의 레시피만 유사도 계산에 사용한다.

In [7]:
recipe_meta = pd.read_csv('data/레시피_메타정보_raw.csv')
recipe_user = pd.read_csv('data/레시피_유저_내용_raw.csv')

## 리뷰 수가 7개 이상, 조회수가 20000회가 넘는 레시피만 사용한다.
inter_recipe__ = [k for k,v in recipe_user['레시피_아이디'].value_counts().items() if v >= 7]
inter_recipe_ = recipe_meta[recipe_meta.조회수 > 20000].레시피_아이디.unique()
inter_recipe = list(set(inter_recipe__) & set(inter_recipe_))  # 약 8000개

### 2. Calculate Cosine Similarity
각 레시피간의 유사도를 계산한다.
- cosine similarity matrix는 대각선을 기준으로 대칭이므로 절반 삼각형만 계산하기로 했다.
    - 이를 위해 combination을 이용했다.
- 각 레시피의 분류 코드들 임베딩들을 가로로 붙였다. (concat)
- 두 레시피 간 카테고리 개수가 다른 경우(e.g. 레시피 A:4가지, 레시피 B:3가지)를 위해 padding을 했다.
    - 넣지 않으면 연산 오류 발생
- 유사도가 0.5 이상 되는 결과만 이용하기로 했다.
    - 약 8000개의 레시피만 가지고 진행했음에도 계산해야 할 유사도가 약 3200만 가지이므로 서버에 무리가 간다.

In [9]:
## 위에서 언급한 일부 레시피에 대해서만 계산 진행 (약 1시간 소요)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

# 유사도 행렬의 절만 삼각형만 계산하기 위해 조합 계산
recipe_combi = list(combinations(inter_recipe,2))  # 32,445,540가지...

# recipe_combi의 element를 key값으로, 그에 대한 유사도 값을 value로 갖는 딕셔너리 (초기화)
sim_dict = {elem : 0 for elem in recipe_combi}

for recipe_id_a, recipe_id_b in recipe_combi:
    # 각 레시피의 카테고리 임베딩 벡터 리스트
    emb_list_a = [catEmbList[cat2idx[code]] for code in recipe_cat_list.loc[recipe_id_a,'분류_코드']]
    emb_list_b = [catEmbList[cat2idx[code]] for code in recipe_cat_list.loc[recipe_id_b,'분류_코드']]

    len_a, len_b = len(emb_list_a), len(emb_list_b)

    # 카테고리 개수가 다르면 padding 처리
    temp = len_a - len_b

    if temp > 0:
        # b padding
        emb_list_b.append(torch.zeros(temp*5).unsqueeze(dim=0))
    elif temp < 0:
        # a padding
        emb_list_a.append(torch.zeros(abs(temp)*5).unsqueeze(dim=0))

    # 리스트에 들어있는 각 임베딩 벡터들을 가로로 붙인다.
    recipe_a = torch.cat(emb_list_a, dim=1).to(device)
    recipe_b = torch.cat(emb_list_b, dim=1).to(device)

    sim_result = F.cosine_similarity(recipe_a, recipe_b).to(device)

    # 유사도가 0.5 이상인 결과만 이용한다.
    # 참고 : cosine similarity는 -1~1 사이의 값을 가진다.
    if sim_result >= 0.5:
        sim_dict[(recipe_id_a, recipe_id_b)] = sim_result

In [10]:
# sim_dict에서 value가 0이 아닌 얘들(즉, 유사도가 0.5 이상)만 추출
# (sim_dict에는 value가 초기화 값 그대로 지정된 순서쌍(즉, 안 볼 것들)도 다 들어가있다.)

simDict_fin = {elem[0] : elem[1] for _, elem in enumerate(sim_dict.items()) if elem[1] != 0}
simDict_fin  # 2,877,090가지  {(3, 4): 200}

{(3375108, 3932168): tensor([0.5169], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881354): tensor([0.8061], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881403): tensor([0.5278], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881447): tensor([0.8122], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6914266): tensor([0.5354], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881542): tensor([0.8122], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881543): tensor([0.8122], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881617): tensor([0.8061], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881637): tensor([0.7938], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881688): tensor([0.7953], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6947288): tensor([0.5169], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 6881800): tensor([0.8959], device='cuda:0', grad_fn=<DivBackward0>),
 (3375108, 4588059): tensor([0.8122], de

### 3. 결과 저장

#### simDict_fin 저장
- 형태 : { (레시피_아이디1, 레시피_아이디2) : 유사도, ... }

In [12]:
# simDict_fin 저장
with open('results/cat_cos_sim.pkl','wb') as fw:
    pickle.dump(simDict_fin, fw)

#### 유사도 큰 상위 N개 레시피 쌍 저장
- 형태 : [ (레시피_아이디1, 레시피_아이디2), (레시피_아이디3, 레시피_아이디4), ... ]

In [26]:
# 상위 1000개의 pair

_, indices = torch.topk(torch.tensor(list(simDict_fin.values())), 1000)

temp = list(simDict_fin.keys())
top1000_pairList = list()

for i in indices:
  top1000_pairList.append(temp[i])

with open("results/top1000_similar_result.csv", 'w') as file:
  writer = csv.writer(file)
  writer.writerows(top1000_pairList)

In [27]:
# 상위 5000개의 pair

_, indices = torch.topk(torch.tensor(list(simDict_fin.values())), 5000)

temp = list(simDict_fin.keys())
top5000_pairList = list()

for i in indices:
  top5000_pairList.append(temp[i])

with open("results/top5000_similar_result.csv", 'w') as file:
  writer = csv.writer(file)
  writer.writerows(top5000_pairList)

In [31]:
# 상위 10000개의 pair

_, indices = torch.topk(torch.tensor(list(simDict_fin.values())), 10000)

temp = list(simDict_fin.keys())
top10000_pairList = list()

for i in indices:
    top10000_pairList.append(temp[i])

with open("results/top10000_similar_result.csv", 'w') as file:
  writer = csv.writer(file)
  writer.writerows(top10000_pairList)

#### 레시피 별 유사한 레시피 리스트 저장
- 인접 리스트와 유사한 형태
- 형태 : { 레시피_아이디1 : [레시피_아이디2, 레시피_아이디3, ...], ... }

In [28]:
# 각 레시피 아이디 별로 유사한 레시피 정보 저장 (~ 인접 리스트)
adjDict_1000 = defaultdict(list)

for recipe_id1, recipe_id2 in top1000_pairList:
    adjDict_1000[recipe_id1].append(recipe_id2)
    adjDict_1000[recipe_id2].append(recipe_id1)

with open('results/top1000_adjDict.pkl','wb') as fw:
    pickle.dump(adjDict_1000, fw)

In [29]:
# 각 레시피 아이디 별로 유사한 레시피 정보 저장 (~ 인접 리스트)
adjDict_5000 = defaultdict(list)

for recipe_id1, recipe_id2 in top5000_pairList:
    adjDict_5000[recipe_id1].append(recipe_id2)
    adjDict_5000[recipe_id2].append(recipe_id1)

with open('results/top5000_adjDict.pkl','wb') as fw:
    pickle.dump(adjDict_5000, fw)

In [33]:
# 각 레시피 아이디 별로 유사한 레시피 정보 저장 (~ 인접 리스트)
adjDict_10000 = defaultdict(list)

for recipe_id1, recipe_id2 in top10000_pairList:
    adjDict_10000[recipe_id1].append(recipe_id2)
    adjDict_10000[recipe_id2].append(recipe_id1)

with open('results/top10000_adjDict.pkl','wb') as fw:
    pickle.dump(adjDict_10000, fw)